# 1. Выбор и преобразование данных для анализа

Подключаю необходимые библиотеки

In [1]:
import pandas as pd, numpy as np
import matplotlib.pyplot as plt, plotly.express as px
from sklearn import tree,ensemble,model_selection,metrics,neighbors
from warnings import filterwarnings as fw
fw('ignore')

Преобразую тестовую и тренировочную выборки: уберу незначащие столбцы, а также преобразую пол в набор бинарных значений. Оставил столбцы Класс обслуживания, Возраст, Цена за билет и Пол (в тренировочной еще бинарный набор выжил/невыжил).

In [2]:
data = pd.read_csv('train.csv')
#data.head()
data=data.drop(columns=['PassengerId','Name','Parch','Ticket','Cabin','Embarked','SibSp']);
#data_train.head()
data['sex1']=np.where(data['Sex']=="male",0,1)
data=data.drop(columns='Sex')
data.head()

,Survived,Pclass,Age,Fare,sex1
0,0,3,22.0,7.2500,0
1,1,1,38.0,71.2833,1
2,1,3,26.0,7.9250,1
3,1,1,35.0,53.1000,1
4,0,3,35.0,8.0500,0


Отберем в отдельные датафреймы характеристики объектов и метки классов.
Затем поделим датасет на обучающую и тестовую выборки в классическом соотношении 1:4.

In [7]:
data=data.dropna()
x = data.iloc[:,1:]
y = data.iloc[:,:1]
xtrain,xtest,ytrain,ytest = model_selection.train_test_split(x,y,test_size=0.2)
print('xtrain: ', xtrain.shape, 'ytrain: ', ytrain.shape, '\nxtest: ', xtest.shape, 'ytest: ', ytest.shape)
ytest=ytest.to_numpy().flatten()

xtrain:  (571, 4) ytrain:  (571, 1) 
xtest:  (143, 4) ytest:  (143, 1)


In [4]:
print(y.shape)

(714, 1)


Проверю наличие NaN для дальнейшей корректной обработки.

In [5]:
for i in data.columns:
    print(data[i].isnull().value_counts())

False    714
Name: Survived, dtype: int64
False    714
Name: Pclass, dtype: int64
False    714
Name: Age, dtype: int64
False    714
Name: Fare, dtype: int64
False    714
Name: sex1, dtype: int64


# 2. Построение модели DecisionTreeClassifier и исследование влияния параметров на качество классификации

In [19]:
DTC = tree.DecisionTreeClassifier()
DTC.fit(xtrain,ytrain)
ypred = DTC.predict(xtest)
print(metrics.classification_report(ytest,ypred,digits=5))
print('AUC (area under ROC curve) = ', metrics.roc_auc_score(ytest,DTC.predict_proba(xtest)[:,1],average='macro',multi_class='ovr'))

              precision    recall  f1-score   support

           0    0.80435   0.81319   0.80874        91
           1    0.66667   0.65385   0.66019        52

    accuracy                        0.75524       143
   macro avg    0.73551   0.73352   0.73447       143
weighted avg    0.75428   0.75524   0.75473       143

AUC (area under ROC curve) =  0.7297125950972104
